# 크롤링한 csv파일을 작업폴더에 넣는다.

In [28]:
#라이브러리를 불러온다
import pandas as pd
import numpy as np
from konlpy.tag import Twitter
from gensim import models

In [29]:
#역 목록을 불러온다
station_list = pd.read_excel('2호선 역 목록.xlsx')

In [30]:
#역 목록을 list로 저장한다.
new_station_list = []
for station in station_list['2호선 역 목록']:
    new_station_list.append(station)
print(new_station_list)

['시청역', '을지로입구역', '을지로3가역', '을지로4가역', '동대문역사문화공원역', '신당역', '상왕십리역', '왕십리역', '한양대역', '뚝섬역', '성수역', '건대입구역', '구의역', '강변역', '잠실나루역', '잠실역', '신천역', '종합운동장역', '삼성역', '선릉역', '역삼역', '강남역', '교대역', '서초역', '방배역', '사당역', '낙성대역', '서울대입구역', '봉천역', '신림역', '신대방역', '구로디지털단지역', '대림역', '신도림역', '문래역', '영등포구청역', '당산역', '합정역', '홍대입구역', '신촌역', '이대역', '아현역', '충정로역', '용답역', '신답역', '용두역', '신설동역', '양천구청역', '신정네거리역', '까치산역']


In [31]:
#포스팅에서 한글 외 문자 제거
import re
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')

In [32]:
w2v_list = []
for station in new_station_list[:50]:
    #역 별로 크롤링 된 파일을 불러와서 저장한다.
    try:
        doc = open(station + '.csv')
        data = doc.read()
        doc.close()               #open했던 파일은 닫는다.
        phrase = data.split('\n') #포스팅 단위로 분리를 해준다.
    except:                      #인코딩 에러가 나서 예외처리를 해준다.
        doc = open(station + '.csv', encoding="utf-8")
        data = doc.read()
        doc.close()
        phrase = data.split('\n')
    # 형태소 분석기
    dic_Twitter = []
    name = doc.name
    for i in range(len(phrase)):
        p = hangul.sub(' ', phrase[i])
        s = Twitter().pos(p, stem=True, norm=True)  ##norm, stem 옵션 추가
        dic_Twitter.append(s)
    #형용사와 명사 단어들을 문장별로 저장 및 특정 단어 제거해서 저장
    dic_ad_noun=[]
    for twit in dic_Twitter:
        c = [x[0] for x in twit if x[1] =="Adjective" or x[1] == "Noun" and len(x[0]) >1 and x[0] not in ["스타","그램",name[:-4],name[:-5],"이다","있다","없다","같다","아니다","그렇다","이렇다","많다","어떻다",
                                             "스럽다","저렇다","인스타그램","맞팔","선팔","이벤트","응모"]]
        dic_ad_noun.append(c)
    #word2vec을 역별로 학습시키고 {역:학습된w2v} 형태의 사전으로 저장
    x=np.str(station)
    y= models.Word2Vec(dic_ad_noun)
    w2v={x:y}
    w2v_list.append(w2v)

In [33]:
#ps_station 이름으로 저장된 형용사 목록을 불러오기 위해 list 생성
ps_st=[]
for i in new_station_list[:50]:
    a="ps_"+ str(i)
    ps_st.append(a)

In [34]:
#ps_station파일에서 형용사을 불러와서 {역이름:형용사} 형태의 사전으로 저장
#형용사 출현 빈도 5 미만인 것은 가져오지 않는다.
ps_dic=[]
for i in ps_st: 
    tag = pd.read_excel(i+'.xlsx')
    print(i)
    s=list(tag.ix[(tag.ix[0:14,3]>5).values,2])
    print(s)
    print(len(s))
    ps_dic.append({i:s})

ps_시청역
['좋다', '맛있다', '덥다', '예쁘다', '이쁘다', '좋아하다', '즐겁다', '행복하다', '힘들다', '고맙다', '배고프다', '시원하다', '귀엽다', '재밌다', '밉다']
15
ps_을지로입구역
['좋다', '특별하다', '맛있다', '예쁘다', '덥다', '이쁘다', '행복하다', '좋아하다', '맵다', '힘들다', '멋지다', '밉다', '아프다', '고맙다', '즐겁다']
15
ps_을지로3가역
['맛있다', '좋다', '예쁘다', '새롭다', '좋아하다']
5
ps_을지로4가역
['맛있다', '좋다']
2
ps_동대문역사문화공원역
['좋다', '아프다', '맛있다', '예쁘다', '이쁘다', '귀엽다', '즐겁다', '덥다', '멋지다', '행복하다', '힘들다', '좋아하다', '아름답다', '멋있다', '고맙다']
15
ps_신당역
['좋다', '맛있다', '행복하다', '예쁘다', '특별하다', '이쁘다', '덥다', '맵다', '즐겁다', '좋아하다', '힘들다', '귀엽다', '밉다', '새롭다', '아쉽다']
15
ps_상왕십리역
['좋다', '맛있다', '이쁘다', '바쁘다', '착하다', '다양하다', '덥다', '즐겁다']
8
ps_왕십리역
['좋다', '맛있다', '예쁘다', '다양하다', '필요하다', '착하다', '이쁘다', '덥다', '행복하다', '즐겁다', '배고프다', '맵다', '밉다', '귀엽다', '좋아하다']
15
ps_한양대역
['좋다', '맛있다', '즐겁다', '예쁘다', '덥다', '고맙다', '따뜻하다', '피곤하다', '새롭다', '재밌다', '이쁘다', '부탁드리다', '싫다', '시원하다', '다양하다']
15
ps_뚝섬역
['좋다', '맛있다', '좁다', '덥다', '예쁘다', '고맙다', '작다', '이쁘다', '좋아하다', '맵다', '즐겁다', '새롭다', '시원하다', '따뜻하다', '건강하다']
15
ps_성수역
['좋다', '맛있다', '좁다', '예쁘다'

In [35]:
#학습된 w2v에 역별 형용사 15개씩 입력하여 나오는 결과 (역,{형용사:결과},{})
w2v_ps_dic=[]
for i, sta in enumerate(new_station_list[:50]):
    try:
        print(i, sta)
        kk=w2v_list[i][sta]
        w2v_dic =[]
        for a in ps_dic[i][ps_st[i]]:
            wps={a:kk.similar_by_word(a,topn=10)}
            w2v_dic.append(wps)
            print(ps_dic[i])
            print(wps)
            print(kk.similar_by_word(a,topn=10))
        w2v_ps_dic.append((sta,w2v_dic))
    except:
        continue
    print("\n")
    print("\n")

0 시청역
{'ps_시청역': ['좋다', '맛있다', '덥다', '예쁘다', '이쁘다', '좋아하다', '즐겁다', '행복하다', '힘들다', '고맙다', '배고프다', '시원하다', '귀엽다', '재밌다', '밉다']}
{'좋다': [('날씨', 0.9150563478469849), ('친구', 0.8799523115158081), ('하루', 0.8613324761390686), ('기분', 0.8608697056770325), ('여유', 0.8537513017654419), ('즐겁다', 0.8460230231285095), ('춥다', 0.8265513181686401), ('너무', 0.8257719874382019), ('퇴근', 0.8189593553543091), ('혼자', 0.8176020383834839)]}
[('날씨', 0.9150563478469849), ('친구', 0.8799523115158081), ('하루', 0.8613324761390686), ('기분', 0.8608697056770325), ('여유', 0.8537513017654419), ('즐겁다', 0.8460230231285095), ('춥다', 0.8265513181686401), ('너무', 0.8257719874382019), ('퇴근', 0.8189593553543091), ('혼자', 0.8176020383834839)]
{'ps_시청역': ['좋다', '맛있다', '덥다', '예쁘다', '이쁘다', '좋아하다', '즐겁다', '행복하다', '힘들다', '고맙다', '배고프다', '시원하다', '귀엽다', '재밌다', '밉다']}
{'맛있다': [('메뉴', 0.9769641160964966), ('존맛', 0.9708682894706726), ('닭도리탕', 0.9600640535354614), ('풍년', 0.9568756818771362), ('수요', 0.9565557241439819), ('불고기', 0.9558503031730652), ('맵다

In [36]:
len(w2v_ps_dic)

50

In [61]:
ps_len=[]
for i in range(50):
    a=len(w2v_ps_dic[i][1])
    ps_len.append(a)
print(ps_len)

[15, 15, 5, 2, 15, 15, 8, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 14, 15, 5, 4, 5, 15, 3, 15, 15]


In [49]:
ps_len[1]

15

In [83]:
#결과 excel파일로 저장하기
for v in range(50):
    #첫번째 형용사와 w2v결과로 DataFrame 생성
    x= pd.DataFrame(w2v_ps_dic[v][1][0])
    z=ps_len[v]-1
    print(new_station_list[v])
    print(z)
    print("==완료==")
    #concat으로 14개의 형용사와 결과 연결하여 붙이기
    for i in range(z):
            print(i+1)
            x= pd.concat([x,pd.DataFrame(w2v_ps_dic[v][1][i+1])], axis=1)    
    #excel로 저장하기
    x.to_excel(new_station_list[v]+'.xlsx')

시청역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
을지로입구역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
을지로3가역
4
==완료==
1
2
3
4
을지로4가역
1
==완료==
1
동대문역사문화공원역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
신당역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
상왕십리역
7
==완료==
1
2
3
4
5
6
7
왕십리역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
한양대역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
뚝섬역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
성수역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
건대입구역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
구의역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
강변역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
잠실나루역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
잠실역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
신천역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
종합운동장역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
삼성역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
선릉역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
역삼역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
강남역
14
==완료==
1
2
3
4
5
6
7
8
9
10
11
12
13
14
교대역
14
==완료==
1
2